In [4]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# batch normalization1
# weights, 
# 训练 测试
# 0.95*prev_mean+0.05*now_mean
# 0.95*prev_std+0.05*now_std

## 模型参数
<img src="./imgs/模型参数.png"  width="700" height="700" align="bottom" />

## 模型参数
<img src="./imgs/bn.png"  width="700" height="700" align="bottom" />

In [5]:

class CharTextCNN(nn.Module):
    def __init__(self,config):
        super(CharTextCNN,self).__init__()
        in_features = [config.char_num] + config.features[0:-1]
        out_features = config.features
        kernel_sizes = config.kernel_sizes
        self.convs = []
        self.conv1 = nn.Sequential(
                    nn.Conv1d(in_features[0], out_features[0], kernel_size=kernel_sizes[0], stride=1), # 一维卷积
                    nn.BatchNorm1d(out_features[0]), # bn层
                    nn.ReLU(), # relu激活函数层
                    nn.MaxPool1d(kernel_size=3, stride=3) #一维池化层
                ) # 卷积+bn+relu+pooling模块
        self.conv2  = nn.Sequential(
            nn.Conv1d(in_features[1], out_features[1], kernel_size=kernel_sizes[1], stride=1),
            nn.BatchNorm1d(out_features[1]),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_features[2], out_features[2], kernel_size=kernel_sizes[2], stride=1),
            nn.BatchNorm1d(out_features[2]),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.Conv1d(in_features[3], out_features[3], kernel_size=kernel_sizes[3], stride=1),
            nn.BatchNorm1d(out_features[3]),
            nn.ReLU()
        )
        self.conv5 = nn.Sequential(
            nn.Conv1d(in_features[4], out_features[4], kernel_size=kernel_sizes[4], stride=1),
            nn.BatchNorm1d(out_features[4]),
            nn.ReLU()
        )
        self.conv6 = nn.Sequential(
            nn.Conv1d(in_features[5], out_features[5], kernel_size=kernel_sizes[5], stride=1),
            nn.BatchNorm1d(out_features[5]),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(8704, 1024), # 全连接层 #((l0-96)/27)*256
            nn.ReLU(),
            nn.Dropout(p=config.dropout) # dropout层
        ) # 全连接+relu+dropout模块

        self.fc2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=config.dropout)
        )

        self.fc3 = nn.Linear(1024, config.num_classes)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)

        x = x.view(x.size(0), -1) # 变成二维送进全连接层
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [6]:
class config:
    def __init__(self):
        self.char_num = 70  # 字符的个数
        self.features = [256,256,256,256,256,256] # 每一层特征个数
        self.kernel_sizes = [7,7,3,3,3,3] # 每一层的卷积核尺寸
        self.dropout = 0.5 # dropout大小
        self.num_classes = 4 # 数据的类别个数

In [7]:
config = config()
chartextcnn = CharTextCNN(config)
test = torch.zeros([64,70,1014])
out = chartextcnn(test)

In [8]:
out

tensor([[-1.8490e-02,  2.0743e-03,  5.4430e-03,  8.0137e-03],
        [-3.5326e-03,  4.5244e-03,  7.8107e-03,  1.2638e-02],
        [-2.1481e-02,  3.4711e-03,  4.8932e-04,  1.1910e-02],
        [-9.8155e-03,  3.5697e-03,  1.2627e-02,  2.7168e-03],
        [-1.6244e-02,  9.2140e-03,  1.9983e-02,  1.2760e-02],
        [-7.2458e-03,  3.3493e-03,  7.4256e-03,  2.3935e-02],
        [-1.7640e-02,  2.1929e-02,  6.5460e-03,  1.2064e-02],
        [ 3.2253e-03,  1.3725e-02, -7.7329e-03,  8.0142e-03],
        [-2.1940e-02,  1.1321e-02,  2.0121e-02,  1.3721e-02],
        [-1.5397e-02,  6.7946e-03,  1.1805e-02,  7.9697e-03],
        [-1.8395e-02,  1.0806e-02, -5.2189e-03,  1.7539e-02],
        [-3.3560e-02,  9.7000e-03,  1.5036e-02,  1.7952e-02],
        [-1.9351e-02, -1.9753e-03,  1.3628e-02,  7.2802e-03],
        [-1.6738e-02,  4.9010e-03,  2.0444e-03,  7.1872e-03],
        [-1.5316e-02, -3.4726e-03,  3.6578e-04,  3.3663e-03],
        [-6.4970e-03,  3.1632e-03,  8.0163e-03,  1.7664e-02],
        

In [9]:
out.shape

torch.Size([64, 4])

In [12]:
from torchsummary import summary

In [16]:
256*(70*7)+256

125696

In [ ]:
1014-7+1

In [13]:
summary(chartextcnn, input_size=(70,1014))
# tensorflow: bn: 256*4

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1            [-1, 256, 1008]         125,696
       BatchNorm1d-2            [-1, 256, 1008]             512
              ReLU-3            [-1, 256, 1008]               0
         MaxPool1d-4             [-1, 256, 336]               0
            Conv1d-5             [-1, 256, 330]         459,008
       BatchNorm1d-6             [-1, 256, 330]             512
              ReLU-7             [-1, 256, 330]               0
         MaxPool1d-8             [-1, 256, 110]               0
            Conv1d-9             [-1, 256, 108]         196,864
      BatchNorm1d-10             [-1, 256, 108]             512
             ReLU-11             [-1, 256, 108]               0
           Conv1d-12             [-1, 256, 106]         196,864
      BatchNorm1d-13             [-1, 256, 106]             512
             ReLU-14             [-1, 2